In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, time
from PIL import Image
import cv2
import glob
import signal

from IPython import display
%matplotlib inline

# Init cameras
## Get available cameras

In [ ]:
def get_cameras(cams=6):
    arr = []
    for index in range(cams):
        print(index)
        cap = cv2.VideoCapture(index)
        if not cap.read()[0]:
            continue
        else:
            arr.append(index)
        cap.release()
    return arr

In [ ]:
cid = get_cameras()
cid 

# Threaded camera reading

In [ ]:
from cameras.threaded_cam import Camera


In [ ]:
cams = [Camera(i, buf_size=10) for i in cid]
ps  = [c.start() for c in cams]


In [ ]:
means = []

while True:
    display.clear_output()
    f, axs = plt.subplots(1,len(cams), figsize=(11,10*len(cams)))
    
    imgs = []
    for c, ax in zip(cams, axs):
        cum, cnt = 0,0
        while not c.buffer.empty():
            im = c.buffer.get() 
            cum += np.mean(im)
            cnt += 1
        mean = cum/cnt
        print(f"Read {cnt} frames from {c.name} with ints {mean}")
        means.append( mean )
        ax.imshow(im)
    plt.pause(0.1)
    f, axs = plt.subplots()
    szs = [ c.buffer.qsize() for c in cams]
    print("buffer sizes:", szs)
    x = np.array(means)
    x = x.reshape(-1, len(cams)).T
    for l  in x:
        axs.plot(l)
    #display.display(plt.show())
plt.show()
        
    

In [ ]:
d = np.array(means).reshape(-1,3).T
plt.plot(d[0][:-50])

In [ ]:
!pwd

In [ ]:
time.strftime('%Y_%m-%d')

In [ ]:
# SAVING TO FILE
save_to="/root/Desktop/quantTechLab/data/auto_rotate/ints1.csv"

In [ ]:
"

print('total number of points: %i'%len(d))
## times = np.arange(0,len(intens),file_check_interval)
data = d
np.savetxt(save_to,data,delimiter=',',fmt='%10.5f')
print ("saved to %s"%save_to)

# Graveyard

Below are snippets that can be usefull for tweaking, but most probably are not needed anymore.

This code rests in peace


In [ ]:
np.array([1,22,33,5,27,39]).reshape(2,3).T

In [ ]:

#
# Attach interrupt handler to release cameras after sigterm
def release_caps(caps):
    for cap in caps:
        try:
            cap.release()
            print("Released camera")
        except Exception as e:
            print("Error",e)
        
def gen_int_handler(caps):
    def int_handler(signum, frame):
        print("Got sigterm")
        release_caps(caps)
        raise  Exception('Interrupt') 
        #cv2.destroyAllWindows()
    return int_handler

def init_caps(idx_list):
    print(f"Init cameras {idx_list}")
    caps = [cv2.VideoCapture(i) for i in (idx_list) ]

    # Set exposition time-------
    for cap in caps:
        if not cap.isOpened():
            print("Failed to init")
        cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, .25)
        # 10 was found to be the smallest value
        cap.set(cv2.CAP_PROP_EXPOSURE, 0.01)
        print("Param::auto exp?",cap.get(cv2.CAP_PROP_AUTO_EXPOSURE))
        print("Param::exp time", cap.get(cv2.CAP_PROP_EXPOSURE))

    signal.signal(signal.SIGINT, gen_int_handler(caps))
    return caps
caps = init_caps(cid)

In [ ]:
release_caps(caps)

In [ ]:

class CameraNoneException(Exception):
    def __init___(self,dErrorArguments):
        Exception.__init__(self,"Camera read None {0}".format(dErrArguments))
        self.dErrorArguments = dErrorArguements
        
        
def read_cameras(caps, idxs, retry_cnt=0):
    results = []
    for camera_idx, cap in zip(idxs,caps):
        print(f"Reading {camera_idx}")
        if cap.isOpened():
            ret, img = cap.read()
        else:
            print("Camera is closed.")
            ret = None
        if not ret:
            print(f"Camera {camera_idx} returned None")

            if retry_cnt<3:
                release_caps(caps)
                time.sleep(0.1)
                print(f"Trying to connect to {camera_idx} only")
                cap_retry = init_caps([camera_idx])
                time.sleep(0.1)
                res_retry = read_cameras(cap_retry, [camera_idx], retry_cnt=retry_cnt+1)
                results += res_retry
            else:
                print(f"!>>\nCamera {camera_idx} is not working at all")
                continue
        else:
            results.append(img)
    return results

while True:
    display.clear_output()
    results = read_cameras(caps, cid)
    print(len(results))
    for res in results:
        plt.imshow(res)
        plt.show()
        plt.close()
    time.sleep(0.5)


In [ ]:
# Main loop

while True:
    for camera_idx, cap in zip(cams_idxs,caps):
        ret, img = cap.read()
        if not ret:
            print(f"Camera {camera_idx} returned None")
            print("Releasing all cameras and trying this camera only")
    if (tries==retry_cnt):
        print("Camera %i is not working. Giving up."%num)
        print()
        break
    time.sleep(retry_sec)


In [ ]:
import cv2
import threading
import time
# Threaded camera reader

class camThread(threading.Thread):
    def __init__(self, img_array, camID, cap):
        threading.Thread.__init__(self)
        self.camID = camID
        self.cap = cap
        self.array = img_array
    def run(self):t
        print ("Starting %s"% self.camID)
        if self.cap.isOpened():  # try to get the first frame
            rval, frame = self.cap.read()
            time.sleep(0)
            if not rval:
                print("fail", self.camID)
                frames = read

        #pic = take_pic(self.camID)
            self.array.append(frame)
        else:
            print("fail")

def take_pic(camID):
    cam = cv2.VideoCapture(camID)
    if cam.isOpened():  # try to get the first frame
        time.sleep(0)
        rval, frame = cam.read()
        time.sleep(0)
        print('read')
        return frame
    else:
        rval = False

# Create two threads as follows
a = []
b = []

thread1 = camThread(a, 0, caps[0])
thread2 = camThread(b, 2, caps[1])
thread1.start()
thread2.start()

In [ ]:
# a library of some unknown buddy from stackoverflow
import cvpubsubs.webcam_pub as w
from cvpubsubs.window_sub import SubscriberWindows

t1 = w.VideoHandlerThread(1)
#t2 = w.VideoHandlerThread(2)
#t3 = w.VideoHandlerThread(3)

t1.start()
#t2.start()
#t3.start()

SubscriberWindows(window_names=['cammy'],
              video_sources=[1]
              ).loop()

t1.join()
#t2.join()
#t3.join()